In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import csv

In [ ]:
import glob
path = "../input/*.csv"
csv_list = []
for fname in glob.glob(path):
    csv_list.append(fname)
    
csv_list

In [ ]:
import sqlite3
import matplotlib.pyplot as plt

In [ ]:
path = "../input/"
database = path + 'database.sqlite'

In [ ]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

In [ ]:
sampledata14 = pd.read_sql("""SELECT *
                        FROM sampledata14;""", conn)
sampledata14.head()

In [ ]:
resultsdata14 = pd.read_sql("""SELECT *
                        FROM resultsdata14;""", conn)
resultsdata14.shape

In [ ]:
common_pests_df = pd.read_sql("""SELECT pestcode, COUNT(sample_pk) AS samples
FROM resultsdata14
GROUP BY pestcode
ORDER BY samples DESC;""", conn)
common_pests_df. head(5)

In [ ]:
max_pests_df = pd.read_sql("""SELECT MAX(samples) AS max_count
FROM 
(SELECT pestcode, COUNT(sample_pk) AS samples
FROM resultsdata14
GROUP BY pestcode);""", conn)
max_pests_df

In [ ]:
common_pests_df = pd.read_sql("""
SELECT B.*
FROM (
    SELECT pestcode, COUNT(sample_pk) AS samples
    FROM resultsdata14
    GROUP BY pestcode
    ORDER BY samples DESC) B
JOIN
    (SELECT MAX(samples) AS max_count
    FROM 
        (SELECT pestcode, COUNT(sample_pk) AS samples
        FROM resultsdata14
        GROUP BY pestcode)) A
ON A.max_count=B.samples
;""", conn)

common_pests_df #What are the most common types of pesticides tested in this study? ANWSER: 15 FROM QUERY

In [ ]:
state_pests_df = pd.read_sql("""SELECT *
FROM (
    SELECT sampledata14.state, resultsdata14.testclass, COUNT(resultsdata14.testclass) AS test_type_count, RANK() OVER(PARTITION BY sampledata14.state ORDER BY COUNT(resultsdata14.testclass) DESC) AS rank
    FROM sampledata14
    JOIN resultsdata14 ON resultsdata14.sample_pk = sampledata14.sample_pk
    GROUP BY sampledata14.state, resultsdata14.testclass
    ORDER BY sampledata14.state ASC) A
WHERE rank <= 3;""", conn)

state_pests_df #Do certain states tend to use one particular pesticide type over another? ANWSER: NO, TYPCIALLY USE "A" PESTICIDE TYPE THE MOST.

In [ ]:
crop_pests_df = pd.read_sql("""SELECT *
FROM (
    SELECT sampledata14.commod, resultsdata14.testclass, COUNT(resultsdata14.testclass) AS test_type_count, RANK() OVER(PARTITION BY sampledata14.commod ORDER BY COUNT(resultsdata14.testclass) DESC) AS rank
    FROM sampledata14
    JOIN resultsdata14 ON resultsdata14.sample_pk = sampledata14.sample_pk
    GROUP BY sampledata14.commod, resultsdata14.testclass
    ORDER BY sampledata14.commod ASC) A
WHERE rank <= 5;""", conn)

crop_pests_df #Does pesticide type correspond more with crop type or location (state)? ANWSER: CROP TYPE

In [ ]:
tolerance_codes_df = pd.read_csv(csv_list[9])

tolerance_codes_df['Pest_Commod'] = tolerance_codes_df['Pesticide Code'] + tolerance_codes_df['Commodity Code']
tolerance_codes_df.head()

In [ ]:
results_df = pd.read_sql("""SELECT *
FROM resultsdata14;""", conn)
results_df['Pest_Commod'] = results_df['pestcode'] + results_df['commod']
results_df.head()

In [ ]:
results_EPA_df = results_df.merge(tolerance_codes_df, on='Pest_Commod')
results_EPA_df.head()

In [ ]:
results_EPA_df = results_EPA_df.rename(columns={'EPA Tolerance Value': 'EPA_Tolerance'})
results_EPA_df.head()

In [ ]:
results_EPA_df['EPA_Tolerance'] = pd.to_numeric(results_EPA_df['EPA_Tolerance'], errors='coerce').fillna(10)
results_EPA_df.head()

In [ ]:
results_EPA_df['EPA_concern']  = results_EPA_df['EPA_Tolerance'] - results_EPA_df['lod']
results_EPA_df.head()

In [ ]:
indexNames = results_EPA_df[results_EPA_df['EPA_concern'] >= 0 ].index

results_concern_df = results_EPA_df.drop(indexNames)

results_concern_df 

In [ ]:
results_concern_df.commod.unique() #Are any produce types found to have higher pesticide levels than assumed safe by EPA standards?